# Neural Networks

Sources:

* http://deeplearning.net/
* https://github.com/dennybritz/nn-from-scratch
* http://neuralnetworksanddeeplearning.com/chap2.html

### Training a neural network

What does a 3-layer neural network with one input layer, one hidden layer, and one output layer look like? The number of input nodes is determined by the dimensionality of the data (let's say, 2). The number of output nodes is determined by the number of classes (again, let's say 2). Note: We could do one class outputing 0 or 1, but having an output node for each class is an easier model to extend to more classes

<img src="nn-3-layer-network.png">

Source: https://github.com/dennybritz/nn-from-scratch

We are free to choose the dimensionality of the hidden layer. A larger number of nodes will be able to fit more complex data, but at a higher computational cost and at the risk of overfitting.

We must also choose an _activation function_ for our hidden layer. The funciton transforms the inputs of the layer into its outputs. Common choices are `tanh`, `sigmoid`, and `ReLU`. We will use `tanh` here. A nice property of $\tanh x$ is that its derivative is $1 - \tanh^2 x$. `tanh` and `sigmoid` are very popular in tutorials, but `ReLU` seems to be more popular in "current" networks used for real tasks.

Because we want out network to output probabilities, the activation function for the output layer will be the `softmax` (or "normalized exponential"), which is simply a way of converting raw scores into probabilities. (It is basically a generalization of the logistic function to multiple classes.)

[Wikipedia](https://en.wikipedia.org/wiki/Softmax_function) has more useful information about the softmax. Effectively, it "squashes" a $K$-dimensional vector $\mathbf{z}$ of arbitrary real values into a $K$-dimensional vector of real values in the range $(0, 1)$ that add up to 1.

\begin{equation}
\sigma(\mathbf{z})_j = \frac{e^{z_j}}{\sum_{k=1}^K e^{z_k}}
\end{equation}
for $j = 1, \dots, K$.

In multinomial logistic regression and linear discriminant analysis, the input to the funciton is the result of $K$ distinct linear functions and the predicted probability for class $j$ given a data vector is:
\begin{equation}
P(y = j|\mathbf{x}) = \frac{e^{\mathbf{x}^T \mathbf{w}_j}}{\sum_{k=1}^K e^{\mathbf{x}^T \mathbf{w}_k}}
\end{equation}

### How does the network make predictions?

This network uses _forward propagation_ to make predictions. If $x$ is the two-dimensional input to our network, we compute the prediction $\hat{y}$ as:

\begin{equation}
\begin{split}
z_1 =&~ x W_1 + b_1 \\
a_1 =&~ \tanh(z_1) \\
z_2 =&~ a_1 W_2 + b_2 \\
a_2 =&~ \hat{y} = \text{softmax}(z_2)
\end{split}
\end{equation}

$z_i$ is the input of layer $i$ and $a_i$ is the output of that layer after applying the activation function. The $w_i$ and $b_i$ are parameters of the network that we must learn from training data. The matrix math dictates the dimensionality of these parameters. If there are 50 input nodes, $W_1 \in \mathbb{R}^{2 \times 50}$, $b_1 \in \mathbb{R}^{50}$, $W_2 \in \mathbb{R}^{50 \times 2}$, and $b_2 \in \mathbb{R}^2$.

### How do we "learn" the parameters?

We must minimize the error on the prediction by varying the parameters. We call the function that measures error the _loss function_. A popular choice with softmax output is the [cross-entropy loss function](https://en.wikipedia.org/wiki/Cross_entropy#Cross-entropy_error_function_and_logistic_regression).

The [Cross entropy](https://en.wikipedia.org/wiki/Cross_entropy) is a measure from [information theory](https://en.wikipedia.org/wiki/Information_theory). It measures the average number of bits needed to identify an event drawn from a given set as belonging to that set if a coding scheme is optimized for an "unnatural" probability distribution modeling those events ($q$) as opposed to the correct, "true" distribution for modeling those events ($p$).

The cross entropy for the distributions $p$ and $q$ over a given set of events is:

\begin{equation}
H(p,q) = E_p\left[-\log q\right] = H(p) + D_{KL}(p || q)
\end{equation}

where $H(p)$ is the [entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory) of $p$ and $D_{KL}$ is the [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback–Leibler_divergence) of $q$ from $p$ (also called the relative entropy of p with respect to q).

For _discrete_ $p$ and $q$ this implies:

\begin{equation}
H(p, q) = -\sum_x p(x) \log q(x)
\end{equation}

For _continuous_ distributions it is:

\begin{equation}
H(p, q) = -\int_X p(x) \log q(x)~dx
\end{equation}

Cross entropy is useful for defining loss functions in machine learning and optimization, particularly for [logistic regression](https://en.wikipedia.org/wiki/Logistic_regression). Here, the true probability $p_i$ is the true (observed) label and the distribution $q_i$ is the predicted value of the current model under consideration.

In its most basic form, logistic regression deals with classifying events into one of two classes (0 and 1). The model predicts an output $y \in \left\{0, 1\right\}$ given an input vector $\mathbf{x}$. The probability is modeled with the logistic function (usually):

\begin{equation}
g(z) = \frac{1}{1 + e^{-z}}
\end{equation}

The probability of finding the output $y = 1$ is given by $q_{y=1} = \hat{y} \equiv g(\mathbf{w} \cdot \mathbf{x})$. The vector of weights, $\mathbf{w}$, is learned via some algorithm (such as [gradient descent](https://en.wikipedia.org/wiki/Gradient_descent)). The conjugate probabillity of finding $y = 0$ is given by $q_{y=0} = 1 - \hat{y}$. The true (_observed_) probabilities may then be expressed similarly as $p_{y=1} = y$ and $p_{y=0} = 1 - y$.

If we have $p \in \left\{y, 1-y\right\}$ and $q \in \left\{\hat{y}, 1-\hat{y}\right\}$, we may use cross entropy to get a measure for the similarity between $p$ and $q$ by simply adding up the terms:

\begin{equation}
\begin{split}
H(p, q) =&~ -\sum_i p_i \log q_i \\
=&~ -y \log \hat{y} - (1 -y) \log (1 - \hat{y}) \\
\end{split}
\end{equation}

The typical loss function that one uses in logistic regression is computed by taking the average of all cross-entropies in the sample. If we have $N$ samples labeled by $n = 1, \dots, N$, the loss function is then:

\begin{equation}
\begin{split}
L(\mathbf{w}) =&~ \frac{1}{N} \sum_{n=1}^N H(p_n, q_n) \\
=&~ -\frac{1}{N} \sum_{n=1}^N \left[ y_n \log \hat{y}_n + (1 - y_n) \log (1 - \hat{y}_n) \right] \\
\end{split}
\end{equation}

With $\hat{y}_n \equiv g(\mathbf{w} \cdot \mathbf{x}_n)$ where $g(z)$ is the logistic function, and $y_i$ are the observed probabilities.

Here, learning the parameters for our network means finding the parameters $W_1, b_1, W_2, b_2$ that minimize the error on our training data, where we are defining _error_ using the _loss function_ discussed above. In particular, we will use the _cross-entropy loss_ (since we have a softmax output). 

For $C$ classes, our formulae generalize slightly from what we have above. In particular, for $N$ examples we have:

\begin{equation}
L(y, \hat{y}) = -\frac{1}{N} \sum_{n \in N} \sum_{i \in C} y_{n,i} \log \hat{y}_{n,i}
\end{equation}

where $\hat{y}$ is our prediction and $y$ are the true labels. All we are doing is summing over our training examples and adding to the "loss" if we predicted the incorrect class. In other words, the further our predictions ($\hat{y}$) are away from the true labels ($y$), the larger the loss.

The goal is to find the parameters that minimize the loss function. We will use [gradient descent](http://cs231n.github.io/optimization-1/) to find the minimum. Here we will implement the most basic version of gradient descent - batched gradient descent with a fixed learning rate. Other variations (stochastic gradient descent (SGD) or minibatch descent) usually perform better in paractice. Ideally, one would also [decay the learning rate](http://cs231n.github.io/neural-networks-3/#anneal).

As an input, gradient descent needs the vector of derivatives (gradients) of the loss function with respect to the parameters. We will compute these with the _backpropagation algorithm_, which is a way to efficiently calculate the gradients starting from the output. See discussions [here](http://colah.github.io/posts/2015-08-Backprop/) and [here](http://cs231n.github.io/optimization-2/).

If we apply backpropagation, we find:

\begin{equation}
\begin{split}
\delta_3 =&~ y - \hat{y} \\
\delta_2 =&~ (1 - \tanh^2 z_1) \odot \delta_3 W_{2}^T \\
\frac{\partial L}{\partial W_2} =&~ a_{1}^T \delta_3 \\
\frac{\partial L}{\partial b_2} =&~ \delta_3 \\
\frac{\partial L}{\partial W_1} =&~ x^T \delta_2 \\
\frac{\partial L}{\partial b_1} =&~ \delta_2 \\
\end{split}
\end{equation}

### What about other activation functions?

Note that this is specific to the `tanh` activation. If we use `ReLU` or `sigmoid` activation functions, that of course changes the derivative in back-propagation.

The sigmoid is

\begin{equation}
\sigma(z) = \frac{1}{1 + e^{-z}}
\end{equation}

The derivative of the sigmoid is

\begin{equation}
\sigma'(z) = \sigma(z) (1 - \sigma(z))
\end{equation}

The [rectifier](https://en.wikipedia.org/wiki/Rectifier_(neural_networks)) may be smoothly approxmated as 

\begin{equation}
f(z) = \ln (1 + e^z)
\end{equation}

This function is also known as the _softplus function_. The derivative of the softplus is

\begin{equation}
f'(z) = \frac{1}{1 + e^{-z}} = \sigma(z)
\end{equation}

(I am actually not certaint what "real" frameworks, e.g. Theano / Caffe do with ReLUs, which strictly speaking, do not feature continuous derivatives - do they use the softplus or do they do something else?)

### What about more complex networks?

Note also that these formulae apply only to one hidden layer. In principle, more layers only requires application of the chain rule. In practice, for complex networks, we must rely on automatic differentiation.

Should we decide to expand to two layers, we of course still use _forward propagation_ to make predictions. If $x$ is the two-dimensional input to our network, and we have _two hidden layers_ we compute the prediction $\hat{y}$ as:

\begin{equation}
\begin{split}
z_1 =&~ x W_1 + b_1 \\
a_1 =&~ \tanh(z_1) \\
z_2 =&~ a_1 W_2 + b_2 \\
a_2 =&~ \tanh(z_2) \\
z_3 =&~ a_2 W_3 + b_3 \\
a_3 =&~ \hat{y} = \text{softmax}(z_3) \\
&~
\end{split}
\end{equation}

For computing losses (recall this toy example network doesn't really use the loss to adjust parameters, just the magnitude of the prediction difference), we also include $W_3$ when computing the regularization term.

    W1, W2, W3 = model['W1'], model['W2'], model['W3']                             
    data_loss += reg_lambda/2 * (np.sum(np.square(W1)) +                           
                                 np.sum(np.square(W2)) +                           
                                 np.sum(np.square(W3)))

Backpropagation must also be adjusted. We begin with:

\begin{equation}
\begin{split}
\delta_3 =&~ y - \hat{y} \\
\delta_2 =&~ (1 - \tanh^2 z_1) \odot \delta_3 W_{2}^T \\
\frac{\partial L}{\partial W_2} =&~ a_{1}^T \delta_3 \\
\frac{\partial L}{\partial b_2} =&~ \delta_3 \\
\frac{\partial L}{\partial W_1} =&~ x^T \delta_2 \\
\frac{\partial L}{\partial b_1} =&~ \delta_2 \\
&~
\end{split}
\end{equation}

We need:

\begin{equation}
\begin{split}
\delta_4 =&~ y - \hat{y} \\
\delta_3 =&~ (1 - \tanh^2 z_2) \odot \delta_4 W_{3}^T \\
\frac{\partial L}{\partial W_3} =&~ a_{2}^T \delta_4 \\
\frac{\partial L}{\partial b_3} =&~ \delta_3 \\
\delta_2 =&~ (1 - \tanh^2 z_1) \odot \delta_3 W_{2}^T \\
\frac{\partial L}{\partial W_2} =&~ a_{1}^T \delta_3 \\
\frac{\partial L}{\partial b_2} =&~ \delta_3 \\
\frac{\partial L}{\partial W_1} =&~ x^T \delta_2 \\
\frac{\partial L}{\partial b_1} =&~ \delta_2 \\
&~
\end{split}
\end{equation}

For more on backpropagation, see http://neuralnetworksanddeeplearning.com/chap2.html.